# **Análisis exploratorio de datos (EDA)**

En 

## **Librerías y modulos necesarios**

In [1235]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

warnings.filterwarnings("ignore")
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np

# **Conjunto de datos 1**

In [1236]:
df = pd.read_csv(r"C:\Users\kamac\OneDrive\Desktop\DelitosBarranquilla\comparativo_de_delitos.csv")

## **Contextualización de los datos**

### **Características de los datos**

In [1237]:
df.head()

,Periodo meses comparado,Años comparados,Delito,Casos/denuncias anterior periodo,Casos/denuncias último periodo,Variación %,Variación absoluta,Fuente
0,01 al 31 de enero,2019-2020,Homicidios,25,23,-8%,-2,"POLICIA MEBAR-GRICRI,datos extraidos del balan..."
1,01 al 31 de enero,2019-2020,Lesiones personales o comunes,279,186,-33%,-93,"POLICIA MEBAR-GRICRI,datos extraidos del balan..."
2,01 al 31 de enero,2019-2020,Hurtos a personas,892,944,6%,52,"POLICIA MEBAR-GRICRI,datos extraidos del balan..."
3,01 al 31 de enero,2019-2020,Hurtos a comercios,171,59,-65%,-112,"POLICIA MEBAR-GRICRI,datos extraidos del balan..."
4,01 al 31 de enero,2019-2020,Hurtos a residencias,85,50,-41%,-35,"POLICIA MEBAR-GRICRI,datos extraidos del balan..."


In [1238]:
df.tail()

,Periodo meses comparado,Años comparados,Delito,Casos/denuncias anterior periodo,Casos/denuncias último periodo,Variación %,Variación absoluta,Fuente
422,1 enero al 31 de diciembre,2022-2023,Hurto a Personas,"11,596","12,902",11%,1306,"POLICIA MEBAR-GRICRI,datos extraidos del balan..."
423,1 enero al 31 de diciembre,2022-2023,Hurto a Comercio,"1,559","1,027",-34%,-532,"POLICIA MEBAR-GRICRI,datos extraidos del balan..."
424,1 enero al 31 de diciembre,2022-2023,Hurto a Residencias,499,661,32%,162,"POLICIA MEBAR-GRICRI,datos extraidos del balan..."
425,1 enero al 31 de diciembre,2022-2023,Hurto Motocicletas,989,"1,168",18%,179,"POLICIA MEBAR-GRICRI,datos extraidos del balan..."
426,1 enero al 31 de diciembre,2022-2023,Hurto a Automotores,481,429,-11%,-52,"POLICIA MEBAR-GRICRI,datos extraidos del balan..."


In [1239]:
df.shape

(427, 8)

In [1240]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427 entries, 0 to 426
Data columns (total 8 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Periodo meses comparado            427 non-null    object
 1   Años comparados                    427 non-null    object
 2   Delito                             427 non-null    object
 3   Casos/denuncias  anterior periodo  427 non-null    object
 4   Casos/denuncias último periodo     427 non-null    object
 5   Variación %                        427 non-null    object
 6   Variación absoluta                 427 non-null    object
 7   Fuente                             427 non-null    object
dtypes: object(8)
memory usage: 26.8+ KB


## **Transformación, limpieza y reorganización del conjunto de datos**

### **Reorganización de variables categorías**

In [1241]:
df.drop('Fuente', axis = 1, inplace = True)

In [1242]:
# Dividir la columna 'Años comparados' en 'Año anterior' y 'Último año'
df[['Año anterior', 'Último año']] = df['Años comparados'].str.split('-', expand = True)

# Eliminar la columna 'Años comparados' original
df.drop('Años comparados', axis = 1, inplace = True)

# Reorganizar las columnas para que las nuevas columnas estén en el lugar adecuado
df = df[['Periodo meses comparado', 'Año anterior', 'Último año', 'Delito', 
         'Casos/denuncias  anterior periodo', 'Casos/denuncias último periodo', 
         'Variación %', 'Variación absoluta']]

In [1243]:
df.rename(columns = {
    'Periodo meses comparado': 'Meses comparados',
    'Casos/denuncias  anterior periodo': 'Casos anterior año',
    'Casos/denuncias último periodo': 'Casos último año',
    'Variación %': 'Variacion porcentual',
    'Variación absoluta': 'Variacion absoluta'
}, inplace = True)


In [1244]:
# Aplicar la transformación usando lambda para agrupar las categorías repetidas
df['Delito'] = df['Delito'].apply(lambda x: 'Delitos sexuales' if 'delito' in str(x).lower() or 'sexuales' in str(x).lower() else (
    'Hurtos a comercios' if 'hurto a comer' in str(x).lower() else (
    'Hurtos a personas' if 'hurto a persona' in str(x).lower() or 'hurto a personas' in str(x).lower() else (
    'Hurtos de motocicletas' if 'hurto motocicletas' in str(x).lower() or 'hurto a motocicletas' in str(x).lower() or 'hurtos motocicletas' in str(x).lower() else (
    'Hurtos a residencias' if 'hurto a residencias' in str(x).lower() else (
    'Hurtos automotores' if 'hurto automotores' in str(x).lower() or 'hurto a automotores' in str(x).lower() else (
    'Lesiones personales o comunes' if 'lesiones personales' in str(x).lower() else (
    'Extorsión' if 'extorsión' in str(x).lower() else (
    'Violencia intrafamiliar' if 'violencia intrafamiliar' in str(x).lower() else (
    'Hurtos a entidades financieras' if 'hurto a entidades financieras' in str(x).lower() or 'hurto a entidad financiera' in str(x).lower() else (
    'Hurtos a residencias' if 'hurto a residencia' in str(x).lower() else (
    'Hurtos de bicicletas' if 'hurto a bicicletas' in str(x).lower() else (
    'Hurtos de celulares' if 'hurto a celulares' in str(x).lower() else x)))))))))))))

In [1245]:
df.value_counts('Delito')

Delito
Homicidios                        47
Hurtos a comercios                47
Hurtos a personas                 47
Hurtos a residencias              47
Hurtos automotores                47
Hurtos de motocicletas            47
Lesiones personales o comunes     47
Hurtos a entidades financieras    32
Extorsión                         26
Delitos sexuales                  15
Violencia intrafamiliar           15
Hurtos de bicicletas               5
Hurtos de celulares                5
Name: count, dtype: int64

In [1246]:
# Filtrar las filas donde haya "ND" en cualquier columna
observaciones_nd = df[df.apply(lambda x: x.str.contains('ND', na = False).any(), axis = 1)]
observaciones_nd

,Meses comparados,Año anterior,Último año,Delito,Casos anterior año,Casos último año,Variacion porcentual,Variacion absoluta
293,1 enero al 31 de octubre,2021,2022,Extorsión,ND,ND,ND,ND
295,1 enero al 31 de octubre,2021,2022,Delitos sexuales,ND,ND,ND,ND
296,1 enero al 31 de octubre,2021,2022,Violencia intrafamiliar,ND,ND,ND,ND
302,1 enero al 31 de octubre,2021,2022,Hurtos a entidades financieras,ND,ND,ND,ND
303,1 enero al 31 de octubre,2021,2022,Hurtos de celulares,ND,ND,ND,ND
304,1 enero al 31 de octubre,2021,2022,Hurtos de bicicletas,ND,ND,ND,ND
306,1 enero al 30 de noviembre,2021,2022,Extorsión,ND,ND,ND,ND
308,1 enero al 30 de noviembre,2021,2022,Delitos sexuales,ND,ND,ND,ND
309,1 enero al 30 de noviembre,2021,2022,Violencia intrafamiliar,ND,ND,ND,ND
314,1 enero al 30 de noviembre,2021,2022,Hurtos automotores,ND,ND,ND,ND


In [1247]:
# Reemplazar "ND" por NaN
df.replace('ND', np.nan, inplace = True)

# Eliminar columnas que contienen solo NaN (es decir, donde todos los valores son NaN)
df.dropna(axis = 1, how = 'all', inplace = True)

In [1248]:
df.describe()

,Meses comparados,Año anterior,Último año,Delito,Casos anterior año,Casos último año,Variacion porcentual,Variacion absoluta
count,427,427,427,427,409,409,409,409
unique,23,4,4,13,344,337,144,293
top,01 al 31 de enero,2021,2022,Homicidios,1,1,0%,0
freq,35,142,142,47,12,8,14,13


## **Delitos**

In [1249]:
df_homicidios = df[df['Delito'] == 'Homicidios']
# Extraer solo el mes de "Meses comparados" tomando la última palabra
df_homicidios['Mes'] = df_homicidios['Meses comparados'].str.split().str[-1]

# Convertir la primera letra de cada mes a mayúscula
df_homicidios['Mes'] = df_homicidios['Mes'].str.capitalize()

# Convertir las columnas de "Casos" a valores numéricos
df_homicidios['Casos anterior año'] = pd.to_numeric(df_homicidios['Casos anterior año'], errors='coerce')
df_homicidios['Casos último año'] = pd.to_numeric(df_homicidios['Casos último año'], errors='coerce')

# Crear una nueva columna con la diferencia de los casos mensuales para el año anterior
df_homicidios['Casos mensuales anterior año'] = df_homicidios['Casos anterior año'].diff().fillna(df_homicidios['Casos anterior año'])

# Crear una nueva columna con la diferencia de los casos mensuales para el último año
df_homicidios['Casos mensuales último año'] = df_homicidios['Casos último año'].diff().fillna(df_homicidios['Casos último año'])

# Agrupar por "Año anterior" para asegurar que la diferencia se calcule correctamente dentro de cada año
df_homicidios['Casos mensuales anterior año'] = df_homicidios.groupby('Año anterior')['Casos anterior año'].diff().fillna(df_homicidios['Casos anterior año'])
df_homicidios['Casos mensuales último año'] = df_homicidios.groupby('Año anterior')['Casos último año'].diff().fillna(df_homicidios['Casos último año'])

# Eliminar las columnas que no son necesarias para el análisis
df_homicidios.drop(columns=['Meses comparados', 'Casos anterior año', 'Casos último año'], inplace = True)

# Renombrar las columnas para que sean más descriptivas
df_homicidios.rename(columns={
    'Casos mensuales anterior año': 'Casos Mensuales Año Anterior',
    'Casos mensuales último año': 'Casos Mensuales Último Año'
}, inplace=True)
# Calcular la variación absoluta
df_homicidios['Variacion absoluta'] = df_homicidios['Casos Mensuales Último Año'] - df_homicidios['Casos Mensuales Año Anterior']

# Calcular la variación porcentual
df_homicidios['Variacion porcentual'] = ((df_homicidios['Casos Mensuales Último Año'] - df_homicidios['Casos Mensuales Año Anterior']) / df_homicidios['Casos Mensuales Año Anterior']) * 100
df_homicidios['Variacion porcentual'] = df_homicidios['Variacion porcentual'].round(1)

# Reubicar las columnas en el orden que prefieras
df_homicidios = df_homicidios[['Mes', 'Año anterior', 'Último año', 'Delito', 'Casos Mensuales Año Anterior', 'Casos Mensuales Último Año', 'Variacion porcentual', 'Variacion absoluta']]

# Seleccionamos las columnas numéricas (excepto las categóricas y las de los años)
numerical_columns = df_homicidios.select_dtypes(include=['float64']).columns

# Convertimos esas columnas a enteros (eliminando los decimales)
df_homicidios[numerical_columns] = df_homicidios[numerical_columns].astype(int)

df_homicidios.to_csv(r'C:\Users\kamac\OneDrive\Desktop\DelitosBarranquilla\homicidios_ordenados.csv', index=False)

In [1250]:
df_homicidios = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\homicidios_ordenados.csv")
df_hurtos_comercios = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\hurtos_comercios_ordenados.csv")
df_hurtos_personas = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\hurtos_personas_ordenados.csv")
df_hurtos_residencias = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\hurtos_residencias_ordenados.csv")
df_hurtos_automotores = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\hurtos_automotores_ordenados.csv")
df_hurtos_motocicletas = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\hurtos_motocicletas_ordenados.csv")
df_lesiones_personales = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\lesiones_personales_ordenados.csv")
df_extorsion = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\extorsion_ordenados.csv")
df_delitos_sexuales = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\delitos_sexuales_ordenados.csv")
df_violencia_intrafamiliar  = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\violencia_intrafamiliar_ordenados.csv")
df_hurtos_bicicletas = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\hurtos_bicicletas_ordenados.csv")
df_hurtos_celulares = pd.read_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\hurtos_celulares_ordenados.csv")

In [1251]:
# Concatenar todos los DataFrames en uno solo
df_total = pd.concat([
    df_homicidios,
    df_hurtos_comercios,
    df_hurtos_personas,
    df_hurtos_residencias,
    df_hurtos_automotores,
    df_hurtos_motocicletas,
    df_lesiones_personales,
    df_extorsion,
    df_delitos_sexuales,
    df_violencia_intrafamiliar,
    df_hurtos_bicicletas,
    df_hurtos_celulares
], ignore_index=True)


df_total.to_csv("C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\todos_delitos_ordenados.csv", index=False)

In [1252]:
# Eliminar filas con valores NaN
df_total = df_total.dropna()

# Seleccionar las columnas numéricas
numerical_columns = df_total.select_dtypes(include=['float64']).columns

# Convertir las columnas numéricas a enteros, eliminando los decimales
df_total[numerical_columns] = df_total[numerical_columns].astype(int)

# Exportar el archivo CSV
df_total.to_csv(r'C:\\Users\\kamac\\OneDrive\\Desktop\\DelitosBarranquilla\\todos_delitos_ordenados.csv', index=False)



In [1253]:
ruta_archivo = r'C:\Users\kamac\OneDrive\Desktop\DelitosBarranquilla\todos_delitos_ordenados.csv'
df_total = pd.read_csv(ruta_archivo)

In [1254]:
df_total.shape

(330, 8)

In [1255]:
df_filtered = df_total[['Mes', 'Año anterior', 'Delito', 'Casos Mensuales Año Anterior']]
df_2023 = df_total[df_total['Último año'] == 2023]
df_2023 = df_2023[['Mes', 'Último año','Delito', 'Casos Mensuales Último Año']]

In [1256]:
df_2023.rename(columns={
    'Mes': 'Mes',
    'Último año': 'Año anterior',
    'Delito': 'Delito',
    'Casos Mensuales Último Año': 'Casos Mensuales Año Anterior'
}, inplace=True)

In [1257]:
df_final = pd.concat([df_filtered, df_2023], ignore_index = True)

In [1258]:
df_final['Casos Mensuales Año Anterior'] = df_final['Casos Mensuales Año Anterior'].apply(lambda x: x if x >= 0 else 0)

In [1259]:
df_final.rename(columns={
    'Mes': 'Mes',
    'Año anterior': 'Año',
    'Delito': 'Delito',
    'Casos Mensuales Año Anterior': 'Cantidad de casos'
}, inplace=True)


In [1260]:
# Crear un mapeo para los meses
meses = {
    "Enero": 1, "Febrero": 2, "Marzo": 3, "Abril": 4, "Mayo": 5,
    "Junio": 6, "Julio": 7, "Agosto": 8, "Septiembre": 9, "Octubre": 10,
    "Noviembre": 11, "Diciembre": 12
}

# Convertir el mes a número usando el mapeo
df_final['Mes_numero'] = df_final['Mes'].map(meses)

# Crear una nueva columna de fecha con el formato Año-Mes
df_final['Fecha'] = pd.to_datetime(df_final['Año'].astype(str) + '-' + df_final['Mes_numero'].astype(str) + '-01')

# Ahora puedes eliminar las columnas adicionales si ya no las necesitas
df_final.drop(columns=['Mes_numero'], inplace=True)

# Reemplazar los valores negativos en 'Cantidad de casos' por 0 (si es necesario)
df_final['Cantidad de casos'] = df_final['Cantidad de casos'].apply(lambda x: max(0, x))

# Crear la columna 'Fecha' con solo el año y el mes
df_final['Fecha'] = df_final['Fecha'].dt.to_period('M')

# Eliminar las columnas originales 'Mes' y 'Año'
df_final.drop(columns=['Mes', 'Año'], inplace=True)

# Reorganizar las columnas, poniendo 'Fecha' al principio
df_final = df_final[['Fecha', 'Delito', 'Cantidad de casos']]

# Exportar el DataFrame df_final a la ruta especificada
df_final.to_csv(r'C:\Users\kamac\OneDrive\Desktop\DelitosBarranquilla\df_final.csv', index=False)

## **Modelo de series de tiempo**

In [1261]:
df_final = pd.read_csv(r'C:\Users\kamac\OneDrive\Desktop\DelitosBarranquilla\df_final.csv')

In [1262]:
df_final.head()

,Fecha,Delito,Cantidad de casos
0,2019-01,Homicidios,25
1,2019-02,Homicidios,24
2,2019-03,Homicidios,25
3,2019-04,Homicidios,20
4,2019-05,Homicidios,20


In [1263]:
df_final.tail()

,Fecha,Delito,Cantidad de casos
406,2023-08,Extorsión,96
407,2023-09,Extorsión,129
408,2023-10,Extorsión,178
409,2023-11,Extorsión,0
410,2023-12,Extorsión,0


In [1264]:
df_hurtos_personas = df_final[df_final['Delito'] == 'Hurtos a personas']

# Mostrar las primeras filas para verificar
df_hurtos_personas.head()

,Fecha,Delito,Cantidad de casos
47,2019-01,Hurtos a personas,892
48,2019-02,Hurtos a personas,924
49,2019-03,Hurtos a personas,1164
50,2019-04,Hurtos a personas,760
51,2019-05,Hurtos a personas,959


In [1265]:
df_hurtos_personas.shape

(57, 3)

In [1266]:
# Asegurarnos de que la columna 'Fecha' esté en formato de fecha, añadiendo el primer día del mes
df_hurtos_personas['Fecha'] = pd.to_datetime(df_hurtos_personas['Fecha'] + '-01', format='%Y-%m-%d')

# Verificar el resultado
df_hurtos_personas.head()


,Fecha,Delito,Cantidad de casos
47,2019-01-01,Hurtos a personas,892
48,2019-02-01,Hurtos a personas,924
49,2019-03-01,Hurtos a personas,1164
50,2019-04-01,Hurtos a personas,760
51,2019-05-01,Hurtos a personas,959


In [1267]:
df_hurtos_personas.tail()

,Fecha,Delito,Cantidad de casos
348,2023-07-01,Hurtos a personas,1707
349,2023-08-01,Hurtos a personas,1218
350,2023-09-01,Hurtos a personas,1026
351,2023-10-01,Hurtos a personas,180
352,2023-11-01,Hurtos a personas,72


In [1268]:

# Asegurarse de que la columna 'Fecha' esté en formato datetime
df_hurtos_personas['Fecha'] = pd.to_datetime(df_hurtos_personas['Fecha'], errors='coerce')

# Lista de meses para realizar las predicciones
meses = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11 ]

# Diccionario para almacenar los resultados de las predicciones
predicciones = []

# Realizar predicciones para cada mes
for mes in meses:
    # Filtrar los datos por mes
    df_mes = df_hurtos_personas[df_hurtos_personas['Fecha'].dt.month == mes]
    df_mes.set_index('Fecha', inplace=True)

    # Ajustar el modelo ARIMA (ajustar los parámetros p, d, q si es necesario)
    modelo_mes = ARIMA(df_mes['Cantidad de casos'], order=(1, 1, 1))  # p=1, d=1, q=1 como ejemplo
    modelo_fit_mes = modelo_mes.fit()

    # Realizar la predicción para el siguiente mes
    forecast_mes = modelo_fit_mes.forecast(steps=1)

    # Crear la fecha de la predicción
    forecast_date_mes = pd.to_datetime(f'2024-{mes:02d}-01')

    # Redondear la predicción al siguiente número entero
    forecast_mes_redondeado = round(forecast_mes[0])

    # Crear un DataFrame con las predicciones
    forecast_df_mes = pd.DataFrame({
        'Fecha': [forecast_date_mes],
        'Delito': ['Hurtos a personas'],  # Aquí puedes cambiar el delito según sea necesario
        'Cantidad de casos': [forecast_mes_redondeado]
    })

    # Almacenar la predicción
    predicciones.append(forecast_df_mes)

# Concatenar todas las predicciones en un solo DataFrame
df_predicciones = pd.concat(predicciones, ignore_index=True)

df_predicciones


,Fecha,Delito,Cantidad de casos
0,2024-01-01,Hurtos a personas,921
1,2024-02-01,Hurtos a personas,1974
2,2024-03-01,Hurtos a personas,800
3,2024-04-01,Hurtos a personas,1373
4,2024-05-01,Hurtos a personas,1192
5,2024-06-01,Hurtos a personas,1796
6,2024-07-01,Hurtos a personas,1776
7,2024-08-01,Hurtos a personas,1225
8,2024-09-01,Hurtos a personas,985
9,2024-10-01,Hurtos a personas,465


In [1269]:
df_modelo = pd.concat([df_hurtos_personas, df_predicciones], ignore_index=True)

In [1270]:
# Asegurarse de que la columna 'Fecha' esté en formato datetime
df_modelo['Fecha'] = pd.to_datetime(df_modelo['Fecha'], errors='coerce')

# Lista de meses para realizar las predicciones para 2025
meses = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # Meses de todo el año

# Diccionario para almacenar los resultados de las predicciones
predicciones_2025 = []

# Realizar predicciones para cada mes de 2025
for mes in meses:
    # Filtrar los datos por mes
    df_mes = df_modelo[df_modelo['Fecha'].dt.month == mes]
    df_mes.set_index('Fecha', inplace=True)

    # Ajustar el modelo ARIMA (ajustar los parámetros p, d, q si es necesario)
    modelo_mes = ARIMA(df_mes['Cantidad de casos'], order=(1, 1, 1))  # p=1, d=1, q=1 como ejemplo
    modelo_fit_mes = modelo_mes.fit()

    # Realizar la predicción para el siguiente mes
    forecast_mes = modelo_fit_mes.forecast(steps=1)

    # Crear la fecha de la predicción para el año 2025
    forecast_date_mes = pd.to_datetime(f'2025-{mes:02d}-01')

    # Redondear la predicción al siguiente número entero
    forecast_mes_redondeado = round(forecast_mes[0])

    # Crear un DataFrame con las predicciones
    forecast_df_mes = pd.DataFrame({
        'Fecha': [forecast_date_mes],
        'Delito': ['Hurtos a personas'],  # Aquí puedes cambiar el delito según sea necesario
        'Cantidad de casos': [forecast_mes_redondeado]
    })

    # Almacenar la predicción
    predicciones_2025.append(forecast_df_mes)

# Concatenar todas las predicciones de 2025 en un solo DataFrame
df_predicciones_2025 = pd.concat(predicciones_2025, ignore_index=True)

# Mostrar las predicciones de 2025
df_predicciones_2025


,Fecha,Delito,Cantidad de casos
0,2025-01-01,Hurtos a personas,921
1,2025-02-01,Hurtos a personas,1954
2,2025-03-01,Hurtos a personas,908
3,2025-04-01,Hurtos a personas,1373
4,2025-05-01,Hurtos a personas,172
5,2025-06-01,Hurtos a personas,1796
6,2025-07-01,Hurtos a personas,1768
7,2025-08-01,Hurtos a personas,1229
8,2025-09-01,Hurtos a personas,971
9,2025-10-01,Hurtos a personas,555


In [1271]:
df_modelos = pd.concat([df_modelo, df_predicciones_2025], ignore_index=True)

In [1272]:
df_modelos.to_csv(r'C:\Users\kamac\OneDrive\Desktop\DelitosBarranquilla\predicciones_hurtos_2024_2025.csv', index=False)


# **Conjunto de datos 2**

In [1273]:
data = pd.read_csv(r"C:\Users\kamac\OneDrive\Desktop\DelitosBarranquilla\hurtos_colombia.csv")
data2 = pd.read_csv(r"C:\Users\kamac\OneDrive\Desktop\DelitosBarranquilla\hurtos2_colombia.csv")

In [1274]:
# Asegurarse de que la columna 'FECHA HECHO' esté en formato datetime
data['FECHA HECHO'] = pd.to_datetime(data['FECHA HECHO'], format='%d/%m/%Y')
data = data[['FECHA HECHO', 'DEPARTAMENTO', 'MUNICIPIO', 'TIPO DE HURTO', 'CANTIDAD']]
# Filtrar los datos por años mayores o iguales a 2019
data_2019 = data[data['FECHA HECHO'].dt.year == 2019]
data_2020 = data[data['FECHA HECHO'].dt.year == 2020]
data_2021 = data[data['FECHA HECHO'].dt.year == 2021]
data_2022 = data[data['FECHA HECHO'].dt.year == 2022]
data_2023 = data[data['FECHA HECHO'].dt.year == 2023]
data_final = pd.concat([data_2019, data_2020, data_2021, data_2022, data_2023], ignore_index=True)

In [1275]:
# Asegurarse de que la columna 'FECHA HECHO' esté en formato datetime
data2['FECHA HECHO'] = pd.to_datetime(data2['FECHA HECHO'], format='%d/%m/%Y')
data2 = data2[['FECHA HECHO', 'DEPARTAMENTO', 'MUNICIPIO', 'TIPO DE HURTO', 'CANTIDAD']]
# Filtrar los datos por años mayores o iguales a 2019
data2_2019 = data2[data2['FECHA HECHO'].dt.year == 2019]
data2_2020 = data2[data2['FECHA HECHO'].dt.year == 2020]
data2_2021 = data2[data2['FECHA HECHO'].dt.year == 2021]
data2_2022 = data2[data2['FECHA HECHO'].dt.year == 2022]
data2_2023 = data2[data2['FECHA HECHO'].dt.year == 2023]
data2_final = pd.concat([data2_2019, data2_2020, data2_2021, data2_2022, data2_2023], ignore_index=True)

In [1276]:
data_f  = pd.concat([data_final, data2_final], ignore_index = True)

In [1277]:
data_f.shape

(474917, 5)

In [1278]:
data_f.drop(columns=['MUNICIPIO'], inplace=True)
data_f


,FECHA HECHO,DEPARTAMENTO,TIPO DE HURTO,CANTIDAD
0,2019-01-01,VALLE,HURTO AUTOMOTORES,2
1,2019-01-01,VALLE,HURTO AUTOMOTORES,3
2,2019-01-01,VALLE,HURTO AUTOMOTORES,1
3,2019-01-01,VALLE,HURTO AUTOMOTORES,1
4,2019-01-01,ANTIOQUIA,HURTO AUTOMOTORES,1
...,...,...,...,...
474912,2023-12-07,TOLIMA,HURTO ENTIDADES COMERCIALES,1
474913,2023-12-17,VALLE,HURTO ENTIDADES COMERCIALES,1
474914,2023-09-04,VALLE,HURTO ENTIDADES COMERCIALES,1
474915,2023-05-25,VALLE,HURTO ENTIDADES COMERCIALES,1


In [1279]:
data_f.value_counts('TIPO DE HURTO')

TIPO DE HURTO
HURTO RESIDENCIAS              180788
HURTO ENTIDADES COMERCIALES    128064
HURTO MOTOCICLETAS             127780
HURTO AUTOMOTORES               38285
Name: count, dtype: int64

In [1280]:
data_f['TIPO DE HURTO'] = 'Hurtos'
data_f['TIPO DE HURTO'].unique()


array(['Hurtos'], dtype=object)

In [1281]:
data_f.head()

,FECHA HECHO,DEPARTAMENTO,TIPO DE HURTO,CANTIDAD
0,2019-01-01,VALLE,Hurtos,2
1,2019-01-01,VALLE,Hurtos,3
2,2019-01-01,VALLE,Hurtos,1
3,2019-01-01,VALLE,Hurtos,1
4,2019-01-01,ANTIOQUIA,Hurtos,1


In [1282]:
# Asegurarse de que la columna 'FECHA HECHO' esté en formato datetime
data_f['FECHA HECHO'] = pd.to_datetime(data_f['FECHA HECHO'])

# Crear una columna de 'Año-Mes' para agrupar por mes y año
data_f['Año-Mes'] = data_f['FECHA HECHO'].dt.to_period('M')

# Agrupar por 'Año-Mes', 'Municipio', y 'Tipo de Hurto' y sumar los 'Cantidad de casos'
data_agrupada = data_f.groupby(['Año-Mes', 'DEPARTAMENTO', 'TIPO DE HURTO'])['CANTIDAD'].sum().reset_index()

# Ver los primeros registros después de agrupar
data_agrupada.head()


,Año-Mes,DEPARTAMENTO,TIPO DE HURTO,CANTIDAD
0,2019-01,AMAZONAS,Hurtos,29
1,2019-01,ANTIOQUIA,Hurtos,2094
2,2019-01,ARAUCA,Hurtos,69
3,2019-01,ATLÁNTICO,Hurtos,564
4,2019-01,BOLÍVAR,Hurtos,425


In [1283]:
data_agrupada.tail()

,Año-Mes,DEPARTAMENTO,TIPO DE HURTO,CANTIDAD
1906,2023-12,SUCRE,Hurtos,48
1907,2023-12,TOLIMA,Hurtos,166
1908,2023-12,VALLE,Hurtos,851
1909,2023-12,VAUPÉS,Hurtos,6
1910,2023-12,VICHADA,Hurtos,4


In [1284]:
data_agrupada.shape

(1911, 4)

In [1285]:
data_agrupada['DEPARTAMENTO'] = data_agrupada['DEPARTAMENTO'] + ' - COLOMBIA'

In [1286]:
# Exportar el dataframe agrupado a un archivo CSV
data_agrupada.to_csv(r'C:\Users\kamac\OneDrive\Desktop\DelitosBarranquilla\hurto_agrupado.csv', index=False)
